# Experiment Architecture
## RAG (Retrieval Augmented Generation)
#### Differences
- Delta is not used for data operations in this experiment
- The input data are PDF not XML files
#### Problem Statement
- We want to get Dolly knowledgeable about Leader Election in Distributed Computing Systems
#### Flow
<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/product/llm-dolly/llm-dolly-full.png" width="1000" />


# Reading PDF papers
##### All the downloaded academic papers are about "Leader Election in Distributed Computing Systems"
The papers have been downloaded from **arXiv**

<img src="https://info.arxiv.org/brand/images/brand-logo-primary.jpg" width="250" />


In [18]:
import PyPDF2
from tqdm import tqdm
import os


def divide_chunks(text_body, min_chunk_size=300, max_chunk_size=1000):
    in_chunks = text_body.split(".\n")
    out_chunks = []
    curr_chunk = ""
    for ic in in_chunks:
        if len(ic) < min_chunk_size:
            if len(ic)+len(curr_chunk) < max_chunk_size:
                curr_chunk = curr_chunk + "\n" + ic
            else:
                out_chunks.append(curr_chunk)
                curr_chunk = ""
    return out_chunks


def clean_chunk(raw_chunk):
    chunk = raw_chunk.replace('-\n','').replace('\n',' ')
    return chunk


pdf_paths = [x for x in os.listdir("data/pdf/leader_election") if ".pdf" in x]

print("\n".join(pdf_paths))

bodies = []
titles = []

for pdf_path in pdf_paths:
    with open(f"data/pdf/leader_election/{pdf_path}", "rb") as file:
        reader = PyPDF2.PdfReader(file)
        body = ""
        for page in tqdm(reader.pages):
            text = page.extract_text()
            body += "\n\n" + text
        bodies.append(body.strip())
        titles.append(pdf_path.strip())

chunk_list = []
for body in bodies:
    chunks = [clean_chunk(c) for c in divide_chunks(body)[1:]]
    chunk_list.append(chunks)

Modified Bully Algorithm using Election.pdf
Improved Bully Election Algorithm for Distributed Systems.pdf
Improved Tradeoffs for Leader Election.pdf
ZePoP A Distributed Leader Election Protocol using the Delay-based Closeness Centrality for Peer-to-Peer Applications.pdf
A Survey and Taxonomy of Leader Election Algorithms in Distributed Systems.pdf
Distributed Consensus in Content Centric Networking.pdf
Fault Toleran Leader Election in Distributed Systems.pdf


100%|██████████| 8/8 [00:00<00:00, 26.78it/s]

# Embeddings
##### Multidimensional Vector Representation of Semantic Meanings
<img src="https://corpling.hypotheses.org/files/2018/04/Screen-Shot-2018-04-25-at-13.21.44.png" width="400" />


In [ ]:
from sentence_transformers import SentenceTransformer
import itertools

model_name = "sentence-transformers/all-MiniLM-L12-v2"

model = SentenceTransformer(model_name, device='cuda')

sentences = list(itertools.chain(*chunk_list))

embeddings = [[float(x) for x in model.encode(s)] for s in sentences]

# Database
##### Creating a Vector Database of Embeddings using the open-source ChromaDB
<img src="https://www.mlq.ai/content/images/2023/08/1_admwyPyR6v_IZI0EYE--eA-1.webp" width="250" />


In [26]:
import chromadb
from chromadb.utils import embedding_functions


chroma_client = chromadb.Client()

default_ef = embedding_functions.DefaultEmbeddingFunction()

collection = chroma_client.get_or_create_collection(name="leader_election_distributed_systems", embedding_function=default_ef)

collection.add(
    documents=sentences,
    embeddings=embeddings,
    ids=[str(x) for x in range(len(embeddings))]
)

# Base Model
##### The open-source Databricks' Dolly
<img src="https://www.databricks.com/sites/default/files/2023-04/Dolly-logo.png" width="300" />

In [27]:
from transformers import pipeline
import torch
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain


def build_qa_chain():

    model_name = "databricks/dolly-v2-3b" # Dolly smallest version (3 billion params)

    instruct_pipeline = pipeline(model=model_name, torch_dtype=torch.bfloat16, trust_remote_code=True,
                                 return_full_text=True, max_new_tokens=4096, top_p=0.95, top_k=50,
                                 device=0) #cuda

    template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instruction:
    You are an expert about leader election algorithms in distributed systems.
    You use a simple language to explain concepts.
    You reply using only short textual descriptions, no images.

    {context}

    Question: {question}

    Response:
    """

    prompt = PromptTemplate(input_variables=['context', 'question'], template=template)

    hf_pipe = HuggingFacePipeline(pipeline=instruct_pipeline)

    return load_qa_chain(llm=hf_pipe, chain_type="stuff", prompt=prompt, verbose=True)

In [28]:
# Building the chain will load Dolly and can take several minutes depending on the model size
qa_chain = build_qa_chain()

In [30]:
import os
os.environ['CURL_CA_BUNDLE'] = ''

class Document():
    def __init__(self, content):
        self.page_content = content
        self.metadata = {"metadata": ""}

def get_similar_docs(question):
    results = collection.query(
        query_embeddings=[float(x) for x in model.encode(question)],
        n_results=3
    )
    return results["documents"]

def answer_question(question):
    similar_docs = [Document(x) for x in get_similar_docs(question)]
    result = qa_chain({"input_documents": similar_docs, "question": question})
    return result

question = "Why distributed systems need a leader?"

answer = answer_question(question)

print(answer["output_text"])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instruction:
    You are an expert about leader election algorithms in distributed systems.
    You use a simple language to explain concepts.
    You reply using only short textual descriptions, no images.

    [' [14] Shay Kutten,William K MosesJr,GopalPandurangan, and David Peleg. 2020. Singularly OptimalRandomized Leader Election.In 34th InternationalSymposiumonDistributedComputing  [15] Shay Kutten, Gopal Pandurangan, David Peleg, Peter Rob inson, and Amitabh Trehan. 2015. On the complexity of universal leaderelection. Journalof theACM (JACM) 62,1 (2015),1–27 [16] Shay Kutten, Gopal Pandurangan, David Peleg, Peter Rob inson, and Amitabh Trehan. 2015. Sublinear bounds for randomizedleader election. TheoreticalComputerScience 561(2015),134–143 [17] Gérard LeLann. 1977. 

In [31]:
question = "Explain shortly the modified bully algorithm for leader election in distributed systems"

answer = answer_question(question)

print(answer["output_text"])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instruction:
    You are an expert about leader election algorithms in distributed systems.
    You use a simple language to explain concepts.
    You reply using only short textual descriptions, no images.

    [' Bully algorithm was ﬁrst presented by Garcia Molina in 1982. The Bully algorithm in distributed computing system is used for dynamically electing a leader by using the process ID number. The process with the highest process ID number is elected as the leader process 1. Assumptions (i) Each process has a unique and not null number to distinguish them and each process knows other process number (ii) Processes dont know which ones are currently up and down (iii) The entire system is synchronous. Timeouts are used for deciding process failure (iv) Processes can cras

In [33]:
question = "Explain shortly the ZePoP distributed leader election protocol"

answer = answer_question(question)

print(answer["output_text"])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Instruction:
    You are an expert about leader election algorithms in distributed systems.
    You use a simple language to explain concepts.
    You reply using only short textual descriptions, no images.

    ['  54 P Beaulah Soundarabai, et al., 2. Sandipan Basu. An Eﬃcient Approach of Election Algorithm in Distributed Systems, Indian Journal of Computer Science and Engineering (IJCSE) , 2:16–21, 2011 3. Muhammad Mahbubur Rahman and Afroza Nahar. Modiﬁed Bully Algorithm using Election Commission, MASAUM Journal of Computing(MJC) , 1(3):439–446, October 2009 4. Chang-Young Kim and Sung-Hoon Bauk. The Election Protocol for Reconﬁgurable Distributed Systems, in International Conference on Wireless Networks (ICWN) , pages 295–301, 2006 5. M S Kordafshari, M Gholipour, M Ja